<a href="https://colab.research.google.com/github/kes1492/Korea_NLP_KimEuiSoon/blob/main/%ED%95%9C%EA%B8%80%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D_(BERT_KoBert_Pytorch).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 한글 감성분석 (BERT - KoBert Pytorch)#

### 네이버 NSMC data set 

#####   출처 : 아래 제작자 기본모델 사용
#####   https://github.com/SKTBrain/KoBERT

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
#!pip install transformers
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 55.0MB 103kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 4.4MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588527 sha256=fee81b55f2e3fd22032a867e32381193140832167915cf325b43da48c20e8990
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.1MB 4.2MB/s 
     |████████████████████████████████| 778kB 4.3MB/s 
     |████████████████████████████████| 890kB 36.7MB/s 
     |████████████████████████████████| 3.0MB 17.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=e60041019ee8384fc6d57b4a5956a17eb2260d6572ba39670f529b2b95e8a629
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a501

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-g3fzs6cn
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-g3fzs6cn
  Created wheel for kobert: filename=kobert-0.1.2-cp36-none-any.whl size=12734 sha256=f5bb9bc28013de5125ed5e01a2e13c3e8a77371ad17892fd51fc4843e5b56082
  Stored in directory: /tmp/pip-ephem-wheel-cache-570iufd9/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

import tensorflow as tf

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [ ]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Dec 23 09:54:19 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    23W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")

In [ ]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
#!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
#!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1
import re
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")


('ratings_test.txt', <http.client.HTTPMessage at 0x7f94ed6202b0>)

In [ ]:
#dataset_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
#dataset_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
dataset_train = nlp.data.TSVDataset("ratings_train.txt", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("ratings_test.txt", field_indices=[1,2], num_discard_samples=1)

In [ ]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [ ]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
model = BERTClassifier(bertmodel,  dr_rate=0.2).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
#        out = model(token_ids, segment_ids, valid_length)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


epoch 1 batch id 1 loss 0.20974838733673096 train acc 0.921875
epoch 1 batch id 201 loss 0.0435853935778141 train acc 0.9821983830845771
epoch 1 batch id 401 loss 0.015600661747157574 train acc 0.9832839775561097
epoch 1 batch id 601 loss 0.17296148836612701 train acc 0.9833610648918469
epoch 1 batch id 801 loss 0.1598111242055893 train acc 0.983477684144819
epoch 1 batch id 1001 loss 0.07307177782058716 train acc 0.9832823426573427
epoch 1 batch id 1201 loss 0.009900991804897785 train acc 0.9832951706910907
epoch 1 batch id 1401 loss 0.02024976722896099 train acc 0.9828470735189151
epoch 1 batch id 1601 loss 0.0531318262219429 train acc 0.9826573235477827
epoch 1 batch id 1801 loss 0.01718619279563427 train acc 0.9826138256524153
epoch 1 batch id 2001 loss 0.022035935893654823 train acc 0.982539980009995
epoch 1 batch id 2201 loss 0.08174622803926468 train acc 0.9821884938664244

epoch 1 train acc 0.9820596914106939


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.8979579603580563


epoch 2 batch id 1 loss 0.2538793683052063 train acc 0.90625
epoch 2 batch id 201 loss 0.04499370604753494 train acc 0.9815764925373134
epoch 2 batch id 401 loss 0.01545265968888998 train acc 0.9823098503740648
epoch 2 batch id 601 loss 0.18464887142181396 train acc 0.9819831530782029
epoch 2 batch id 801 loss 0.10609053820371628 train acc 0.9819366416978776
epoch 2 batch id 1001 loss 0.04171133413910866 train acc 0.9821116383616384
epoch 2 batch id 1201 loss 0.011991012841463089 train acc 0.9817990216486261
epoch 2 batch id 1401 loss 0.016927296295762062 train acc 0.9815087437544611
epoch 2 batch id 1601 loss 0.13878072798252106 train acc 0.9811738757026858
epoch 2 batch id 1801 loss 0.05860808119177818 train acc 0.9809654358689617
epoch 2 batch id 2001 loss 0.043691400438547134 train acc 0.9810094952523738
epoch 2 batch id 2201 loss 0.1828899383544922 train acc 0.980683496138119

epoch 2 train acc 0.980477637940842


...


epoch 3 test acc 0.8927229859335039


epoch 4 batch id 1 loss 0.32980841398239136 train acc 0.90625
epoch 4 batch id 201 loss 0.10244614630937576 train acc 0.9701492537313433
epoch 4 batch id 401 loss 0.06517575681209564 train acc 0.9701527431421446
epoch 4 batch id 601 loss 0.1808369755744934 train acc 0.9696859400998337
epoch 4 batch id 801 loss 0.21061083674430847 train acc 0.9700959737827716
epoch 4 batch id 1001 loss 0.024415379390120506 train acc 0.9696865634365635
epoch 4 batch id 1201 loss 0.10179544985294342 train acc 0.9695305995004163
epoch 4 batch id 1401 loss 0.051566362380981445 train acc 0.9691514989293362
epoch 4 batch id 1601 loss 0.07813186198472977 train acc 0.9688866333541537
epoch 4 batch id 1801 loss 0.04967254027724266 train acc 0.9686979455857857
epoch 4 batch id 2001 loss 0.08224285393953323 train acc 0.9688046601699151
epoch 4 batch id 2201 loss 0.0709000825881958 train acc 0.9684873353021354

epoch 4 train acc 0.9682656072241184



epoch 4 test acc 0.8873281649616368


epoch 5 batch id 1 loss 0.39213722944259644 train acc 0.875
epoch 5 batch id 201 loss 0.18432661890983582 train acc 0.9657182835820896
epoch 5 batch id 401 loss 0.04067561402916908 train acc 0.9664900249376559
epoch 5 batch id 601 loss 0.196856290102005 train acc 0.9665921381031614
epoch 5 batch id 801 loss 0.22372402250766754 train acc 0.9671894506866417
epoch 5 batch id 1001 loss 0.05956277251243591 train acc 0.9668144355644356
epoch 5 batch id 1201 loss 0.014943445101380348 train acc 0.9670456910907577
epoch 5 batch id 1401 loss 0.1782473474740982 train acc 0.9669655603140613
epoch 5 batch id 1601 loss 0.0404018871486187 train acc 0.9669444878201124
epoch 5 batch id 1801 loss 0.046400561928749084 train acc 0.9669714741810106
epoch 5 batch id 2001 loss 0.09842363744974136 train acc 0.9672429410294853
epoch 5 batch id 2201 loss 0.15651798248291016 train acc 0.9671882099045889

epoch 5 train acc 0.967487912400455



epoch 5 test acc 0.8898857097186701


# 새로운 문장 테스트

In [ ]:
#from kobert_transformers import get_kobert_model, get_distilkobert_model
#from tokenization_kobert import KoBertTokenizer
#from keras.preprocessing.sequence import pad_sequences
#tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert') # monologg/distilkobert도 동일
#tokenizer = get_tokenizer()

#MAX_LEN = 128

using cached model


In [ ]:
# 입력 데이터 변환

def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
#    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
     input_ids = tokenizer.convert_tokens_to_ids([sentence])
    # 입력 토큰의 최대 시퀀스 길이


    # 토큰을 숫자 인덱스로 변환
  #  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

IndentationError: ignored

In [ ]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [ ]:
!git clone https://github.com/kes1492/Korea_KimEuiSoon_Data.git
!cp ./Korea_KimEuiSoon_Data/* .

Cloning into 'Korea_KimEuiSoon_Data'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 14 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (14/14), done.


In [ ]:
import pandas as pd
import numpy as np
#from nltk import word_tokenize

ko_data = pd.read_csv("ko_data.csv", encoding = 'cp949', sep=',')
ko_data_doc = ko_data['Sentence']
#ko_data_txt = pd.Series.to_string(ko_data_doc)
ko_data_list = pd.Series.to_list(ko_data_doc)

#dataset_eval = nlp.data.TSVDataset(ko_data_txt)
dataset_eval = nlp.data.TSVDataset(ko_data_list, field_indices=[1,2], num_discard_samples=1)
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
data_eval = BERTDataset(dataset_eval, 0, 1, tok, max_len, True, False)
eval_dataloader = torch.utils.data.DataLoader(data_eval, batch_size=batch_size, num_workers=5)


model.eval()
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(eval_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)

FileNotFoundError: ignored

In [ ]:
##############################   예측 TEST   #################################################

import pandas as pd
import numpy as np
#from nltk import word_tokenize

ko_data = pd.read_csv("ko_data.csv", encoding = 'cp949', sep=',')
ko_data_doc = ko_data['Sentence']
#ko_data['Sentence'] = ko_data['Sentence'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
#ko_data['Sentence'] = ko_data['Sentence'].str.replace("[1-9]","") # 숫자  변경
#ko_data['Sentence'] = ko_data['Sentence'].str.replace("[a-z]","") # 영어  변경
#ko_data['Sentence'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
index = 0
predict_value = [] #예측값 저장 리스트
id_value = [] #id 저장 리스트
neg_num = 0
pos_num = 0
doc_len = len(ko_data_doc)

for sentence in ko_data_doc:
  temp_doc = []
  value = 0  
  temp_doc = ko_data_doc[index]
  # 진행사항 progress
  if index % 1000 == 0 and not index == 0: 
     print('  Data index {:>5,}  of  {:>5,} '.format(index,  doc_len))
     
  logit = test_sentences([temp_doc])
  value = np.argmax(logit)
  if value == 1:
    pos_num += 1
  elif value == 0:
    neg_num += 1
  else:
     print('value is error')

  predict_value.append(value)
  index = index + 1

print(pos_num, neg_num)
#id 예측 list만큼 0부터 순차적으로 저장
id_index = 0
for i in range(len(ko_data_doc)):
  id_value.append(id_index)
  id_index = id_index +1

ko_data_doc_out = pd.DataFrame({'Id':pd.Series(id_value), 'Predicted':pd.Series(predict_value)})

df = pd.DataFrame(ko_data_doc_out)
df.to_csv('ko_predicted(KoBERT)(1).csv', index=False)

################################################################################################

TypeError: ignored

In [ ]:
############################ TEST ####################################################
index = 5020
temp_doc = [] 
temp_doc = ko_data_doc[index]

logits = test_sentences([temp_doc])
print(logits)
print(np.argmax(logits))
ko_data_doc[index]

######################################################################################